<img src="notebookplots/Code_of_Conduct.png" alt="Alt Text" width="100%" height="100%">

In [ ]:
from tqdm import tqdm 
import numpy as np 
import ROOT

To install missing packages, do e.g.:
```python
import sys
!{sys.executable} -m pip install numpy --user
```

In [ ]:
from os import path
desy = '/nfs/dust/cms/group/cmsdas2023/DPG-HCAL/'
lxplus = '/eos/cms/store/group/dpg_hcal/comm_hcal/'

if path.exists(desy):
    filepath = desy
elif path.exists(lxplus):
    filepath = lxplus
else:
    print('No input file path found!')

# Session 1: HcalNano

NanoADD format is a plain tree/tuple with simple branches in root file, helping analysis to be performed without complicated CMSSW analyzers.

In [ ]:
%%bash -s "$filepath"
export input_file=$1/RAW/HLTPhysics/RAW/v1/000/366/821/00000/0335a05c-8b31-49e7-99f0-254eef65485e.root
export output_file=my_test_output.root
cmsDriver.py NANO \
    -s RAW2DIGI,RECO,USER:DPGAnalysis/HcalNanoAOD/hcalNano_cff.hcalNanoTask \
    --processName=MyHcalNanoTest \
    --datatier NANOAOD \
    --eventcontent NANOAOD \
    --filein file:$input_file \
    --fileout $output_file \
    -n 10 \
    --nThreads 4 \
    --conditions auto:run3_data_prompt \
    --era Run3 \
    --python_filename my_cms_config.py \
    --no_exec \
    ## --customise DPGAnalysis/HcalNanoAOD/customiseHcalCalib_cff.customiseHcalCalib ## used for testEnablesEcalHcal stream
cmsRun my_cms_config.py

Now let's look into the tree structure of HcalNano format

In [ ]:
file = ROOT.TFile("my_test_output.root")
tree = file.Get("Events")
for i, branch in enumerate(tree.GetListOfBranches()):
    if i>100: break
    print(branch.GetName())

# Session 2: Reconstructed Hits in HCAL
## Introduction
The HCAL detectors were designed to measure the energy of neutral and charged particles originating from quark fragmentation and hadronization. Optical signals produced by particle interactions in the HB and HE scintillators or HF quartz fibers are carried through clear fibers to Silicon PhotoMultipliers (SiPMs) or Photomultiplier Tubes (PMTs), where they are converted to electric signals. Then, the analog signals from the photo-detectors are converted to digital signals by an electric circuit called the QIE (charge Integrator and Encoder). The QIE collects the charge produced by the photo-detectors, integrates it over 25ns interval and digitizes the signal.

## Pulse shape in HCAL
If we look at the pulse shape before it is integrated over 25 ns intervals, it looks like the plot below. This plot shows the fraction of charge at each ns vs time.

<img src="notebookplots/DP-18-018-HE-pulse.png" alt="Alt Text" width="50%" height="50%">

Look how there is almost no signal below 100 ns and suddenly we have a peak starting at 100 ns. Actually, there is no need for us to keep the charge collected at each ns. Since we have collisions every 25 ns, let’s have 25 ns granularity and integrate the charge each 25 ns (this is what QIEs do). We call each 25 ns a time slice.

## Look at the pulse shape in a run
Let’s look at the pulse shape after QIE charge integration in a run taken in 2015 (heavy ion collisions). 

In [ ]:
chain = ROOT.TChain("ExportTree/HcalTree")
chain.Add(filepath+"/CMS_POS/RecHits/HCALTree_run_261395.root")

PulseShape =  ROOT.TProfile("PulseShape", "PulseShape;Time Slice;Charge (fC)", 10,0,10);

TS = [0] * 10
sumQ = 0
ped = 0


for i, event in tqdm(enumerate(chain)):
    if i>100: break
    for j in range(event.PulseCount):
        if abs(event.IEta[j])>27 or ( abs(event.IEta[j])>14 and abs(event.IEta[j])<19): continue
        if event.Charge[j*10+0] == event.Charge[j*10+4] and event.Charge[j*10+4] == event.Charge[j*10+5]: continue

        sumQ=0
        ped=0
        
        for k in range(3): ped+=event.Charge[j*10+k]+event.Pedestal[j*10+k]
        for k in range(3,7): sumQ+=event.Charge[j*10+k]+event.Pedestal[j*10+k]
        
        ped/=3
        sumQ-=4*ped
        
        for k in range(0, 10): TS[k] = event.Charge[j*10+k]+event.Pedestal[j*10+k]-ped
        
        if sumQ<5 or TS[4]<5 or ped<0 or TS[3]<0 or TS[5]<0 or TS[6]<0: continue
        
        for k in range(0, 10): PulseShape.Fill(k, TS[k])
            
c = ROOT.TCanvas("MC_Data","",900,600)
PulseShape.Draw("HIST")
c.Update()
c.Draw()


In [ ]:
%%bash -s "$filepath"
export inputfile=$1/CMS_POS/RecHits/HCALTree_run_261395.root
root -l
.L Pulse_shape.C
Pulse_shape("$inputfile", "PulseShape_1")
.q

In the plot you just made, we have time slices on the x-axis (each time slice is 25 ns). This plot gives you all the charge a QIE has received in each 25 ns in HB. Note that this is a TProfile, meaning that the plot is an average of all QIEs in HB.

Rerun this markdown to reload the generated plot
<img src="PulseShape_1.png" alt="PulseShape" width="80%" height="80%">

Looking at the plot, we always have the peak at TS4. Actually, we time HCAL in a way that the peak of charge from QIEs is always in TS4. (if it arrives earlier, we delay the response of QIE to have the peak at TS4). When HCAL sees a peak, it saves four TS before and five TS after (totally 10 TS from TS0 to TS9) and then corresponds this hit to the right bunch crossing. If it happens that the peak is 25 ns earlier or later (i.e. in TS3 or TS5), HCAL will assign the hit to the wrong bunch crossing. We will come back to this when we talk about out-of-time pile-up.

## Method-0 energy reconstruction
The easiest thing that comes to mind to measure the energy of a hit is to just simply add up all charges from all 10 TS and convert them to energy using the look-up tables (look-up tables are tables that convert fC to GeV). This simplest algorithm to calculate the energy of a hit is called method-0 and was used in run 1 (2009 – 2013) because in run 1, we had collisions every 50 ns instead of 25 ns. We will see why we should be more careful when bunch spacing is shorter.

## Out-of-time Pile-up
Now let’s see what will happen if we have two hits from two consecutive bunch crossings in the same scintillator. In this case the second hit is 25 ns after the first one. But since the response time of HBDs are more than 25 ns, two hits will mix:
<img src="notebookplots/OOT-PU.png" alt="text" width="80%" height="80%">

It looks like method-0 is not very accurate when we have more than one hit. We need a way to recognize pulses with more than one hit and treat them differently. Look how pulse shapes are different when we have only one hit and when we have two hits. The change in pulse shape is what we can use to distinguish pulses with only one hit and pulses with more than one hit.

The pulse shape we looked at in the beginning of this exercise was from a lead-lead collision taken in 2015 with isolated bunches. Therefore, the bunch crossings were far enough not to affect each other. Now let’s look at a run with bunch spacing of 25 ns. It’s a proton-proton collision run taken in 2017. Do:

In [ ]:
%%bash -s "$filepath"
export inputfile=$1/CMS_POS/RecHits/HCALTree_run_301998.root
root -l
.L Pulse_shape.C
Pulse_shape("$inputfile", "PulseShape_2")
.q

Rerun this markdown to reload the generated plot
<img src="PulseShape_2.png" alt="PulseShape" width="80%" height="80%">

Is there any difference between the pulse shape of this run (with 25 ns bunch spacing) and the previous one (isolated bunches)? Why?

## Method-2 / Mahi energy reconstruction
Method-2 uses the pulse shape difference to recognize two hit pulses and tries to find the real energy of the hit we are interested in.

Hits that come from previous or later bunch crossings are called “out-of-time pile-up”. Method-2 compensates for out-of-time pile-up.

Method-2 or Mahi can fit up to three templates to the pulse shape:
First a fit is performed with only one hit template. If the fit matches the pulse shape, method-2 will assume it’s only a single hit.

If the single hit fit doesn’t match the pulse shape, method-2 or mahi will try another template that can consider up to two out-of-time hits (M2) or up to seven out-of-time hits (Mahi).

You can visualize the results of Mahi fit by

In [ ]:
%%bash

root -l -b -q pulsedisplay.C

Rerun this markdown to reload the generated plot  
<div style="display: flex; flex-wrap: wrap; justify-content: space-between;">
    <div style="flex: 48%; padding: 5px;">
        <img src="pulse_run316944_ls20_evt1166071_ieta-25_iphi21_depth2.png" alt="Plot A" style="width:100%">
        <p style="text-align:center;">Plot A</p>
    </div>
    <div style="flex: 48%; padding: 5px;">
        <img src="pulse_run316944_ls20_evt1185262_ieta-17_iphi24_depth2.png" alt="Plot B" style="width:100%">
        <p style="text-align:center;">Plot B</p>
    </div>
</div>
<div style="display: flex; flex-wrap: wrap; justify-content: space-around;">
    <div style="flex: 48%; padding: 5px;">
        <img src="pulse_run316944_ls4_evt184975_ieta1_iphi50_depth1.png" alt="Plot C" style="width:100%">
        <p style="text-align:center;">Plot C</p>
    </div>
    <div style="flex: 48%; padding: 5px;">
        <img src="pulse_run316944_ls20_evt1223228_ieta-23_iphi41_depth2.png" alt="Plot D" style="width:100%">
        <p style="text-align:center;">Plot D</p>
    </div>
</div>
<div style="display: flex; flex-wrap: wrap; justify-content: space-around;">
    <div style="flex: 48%; padding: 5px;">
        <img src="pulse_run316944_ls20_evt1244265_ieta-26_iphi51_depth1.png" alt="Plot E" style="width:100%">
        <p style="text-align:center;">Plot E</p>
    </div>
    <div style="flex: 48%; padding: 5px;">
        <img src="pulse_run316944_ls21_evt2696747_ieta-7_iphi18_depth1.png" alt="Plot F" style="width:100%">
        <p style="text-align:center;">Plot F</p>
    </div>
</div>

## Method-3 energy reconstruction
The last reconstruction method is called Method-3. Method-3 is a lighter version of method-2 that can be used online by fitting 3 TS. it provides similar pile-up mitigation as method-2 with smaller computing time.

## Comparison between different reconstructions methods
In method-2, method-3 and Mahi (unlike method-0) we reconstruct the energy of the hit that we are interested in (i.e. the hit that peaks at TS4) from the pulse shape. Let’s compare the energies reconstructed by method-0, method-2 and method-3. Do:

In [ ]:
file = ROOT.TFile(filepath+"CMS_POS/RecHits/HCALTree_run_301998.root", "READ")
HcalTree = file.Get("ExportTree/HcalTree")
HcalTree.Print()

In [ ]:
HcalTree.Draw("Method2Energy:(Charge[][4]+Charge[][5]+Charge[][6]+Charge[][7])","Method2Energy<500 && Method2Energy>10 && (Charge[][4]+Charge[][5]+Charge[][6]+Charge[][7])<100000","COLZ")
histogram = ROOT.gPad.GetPrimitive("htemp")
histogram.GetXaxis().SetTitle("Method2Energy (GeV)")
histogram.GetYaxis().SetTitle("Method0Energy (GeV)")
ROOT.gPad.Update()
ROOT.gPad.Modified()
ROOT.gPad.GetCanvas().Draw()

In [ ]:
HcalTree.Draw("Method2Energy:Method3Energy >> hist","Method2Energy<500 && Method2Energy>10","COLZ")
histogram = ROOT.gPad.GetPrimitive("hist")
histogram.SetStats(0)
histogram.GetXaxis().SetTitle("Method2Energy (GeV)")
histogram.GetYaxis().SetTitle("Method3Energy (GeV)")
ROOT.gPad.Update()
ROOT.gPad.Modified()
ROOT.gPad.GetCanvas().Draw()

# Section 2: cosmic data analysis
## Introduction
Muons are minimum ionizing particles. One Expect a muon releases about 2 GeV in the HCAL Barrel. In the MWGR3 we instrumented 4 sections of the HB+ (iphi:47-62 and ieta:0-16) and the cosmic triggers are provided by the DT. Each readout unit read more than one layer of scintillator. In cosmic collision the energy of each readout units is estimated with Method 0. Muons are used for inter-channel calibration.

<img src="notebookplots/HCALLayoutMWGR3.png" alt="text" width="80%" height="80%">

To get total hadronic energy (i.e. summed over depths crossed):

In [ ]:
file = ROOT.TFile(filepath+"CMS_POS/cosmic/525486FA-F98F-0849-843E-1E23C444DCB2.root", "READ")
EventTree = file.Get("Events")
#EventTree.Print()
EventTree.Draw("recoMuons_muons__RECO.obj.calEnergy().had >> htemp(100, 0, 10)","recoMuons_muons__RECO.obj.calEnergy().crossedHadRecHits.detId.iphi()>46&&recoMuons_muons__RECO.obj.calEnergy().crossedHadRecHits.detId.iphi()<63&&recoMuons_muons__RECO.obj.calEnergy().crossedHadRecHits.detId.ieta()>0&&recoMuons_muons__RECO.obj.calEnergy().crossedHadRecHits.detId.ieta()<17")
histogram = ROOT.gPad.GetPrimitive("htemp")
histogram.GetXaxis().SetTitle("Deposited Muon Energy (GeV)")
histogram.GetXaxis().SetRangeUser(0, 10)
histogram.GetYaxis().SetTitle("Entries")
ROOT.gPad.Update()
ROOT.gPad.Modified()
ROOT.gPad.GetCanvas().Draw()

To get the energy deposited in depth 3:

In [ ]:
EventTree.Draw("recoMuons_muons__RECO.obj.calEnergy().crossedHadRecHits.energy","recoMuons_muons__RECO.obj.calEnergy().crossedHadRecHits.detId.iphi()>46&&recoMuons_muons__RECO.obj.calEnergy().crossedHadRecHits.detId.iphi()<63&&recoMuons_muons__RECO.obj.calEnergy().crossedHadRecHits.detId.ieta()>0&&recoMuons_muons__RECO.obj.calEnergy().crossedHadRecHits.detId.ieta()<17&&recoMuons_muons__RECO.obj.calEnergy().crossedHadRecHits.detId.depth()==3")

Let's perform the analysis with FWLite module of CMSSW

In [ ]:
import ROOT
import sys
from os import path
from DataFormats.FWLite import Events, Handle

hist = ROOT.TH1F('energy', ';HCAL energy;number of muons', 50, 0, 5)

inputfile = filepath+"CMS_POS/cosmic/525486FA-F98F-0849-843E-1E23C444DCB2.root"

events = Events(inputfile)
muonhandle = Handle('vector<reco::Muon>')
muonlabel = 'muons'

for i, event in tqdm(enumerate(events)):
    if(i%1000==0): print("i=",i)
    event.getByLabel(muonlabel, muonhandle)
    muons = muonhandle.product()
    #https://cmsdoxygen.web.cern.ch/cmsdoxygen/CMSSW_13_2_5/doc/html/df/de3/classreco_1_1Muon.html 
    for j, muon in enumerate(muons):
        depths = []
        for HcalMuonRecHit in muon.calEnergy().crossedHadRecHits: 
            depths.append(HcalMuonRecHit.detId.depth())
        if all(d in depths for d in [1,2,3,4]): hist.Fill(muon.calEnergy().had)


ROOT.gStyle.SetOptFit(1)
c = ROOT.TCanvas('c', 'c', 500, 450)
c.cd()
hist.Draw()
hist.Fit('landau')
c.Update()
c.Draw()


# Session 4: Isotrack Calibration
## Introduction
The real detector geometry differs from the testbeam geometry where calibration factors were extracted for the first time. Therefore, HCAL experts use several methods to recalibrate the detector. One of the recalibration methods is to use Isolated Charged Hadrons. This exercise will briefly describe how this recalibration is done in HCAL.
## Calibratio method
CMS has a very powerful magnet, the momentum measured by the tracker is accurate. The tracker measures the momentum of charged particles through their curvature. If a particle is very high energy, it will appear like a straight line to the tracker. Therefore, the curvature will be very little and the measurement of momentum is not accurate. This is where the measurement of energy by HCAL is more precise than the momentum measured by the tracker. Obviously, HCAL is also used to measure the energy of neutral hadrons that can't be done by the tracker.

Although radiation damages HCAL, it has no effect on the magnetic field, therefore, the energy measured by the tracker can be used as a reference for HCAL calibration. This method of calibration uses “isolated charged hadrons” for calibration because:
- Charged: Obviously, if we want to use the magnetic field to measure the energy of a particle, that particle should have charge.
- Isolated: We require the particle to be isolated from other particles to ensure minimal contamination from unrelated energy depositions.
- Hadron: We would like the particle to deposit its energy in HCAL, therefore the particle should be a hadron.

Low and high energy isolated charged hadrons are not desirable in the calibration. Because:
- HCAL has a non-linear response with respect to the initial energy of the hadrons which is more prominent at lower energies. Therefore, we don’t want to consider low energy hadrons.
- There is only a low number of isolated hadrons with high energy as the energy spectrum falls down with increasing energy.

For this reason, an energy of 50 GeV is chose and only particles with momenta between 40-60 GeV are considered.

### Measurement uncertainties
Now that we know how we want to calibrate HCAL, let’s look at the uncertainties of measuring energy/momentum in HCAL/tracker. There are a few sources of uncertainty measuring the energy of isolated charged hadrons:
- Primary vertex and track reconstruction quality:Tracks are required to have an associated primary vertex and to have the track quality bit set to “high purity”.
- Interaction of hadrons with ECAL:The hadron calorimeter does not measure the entire energy of particles. The particles can interact and deposit a substantial part of their energy in the preceding electromagnetic calorimeter (ECAL) or other dead material before the hadron calorimeter. The depth of ECAL is about one nuclear interaction length and more than half of hadrons undergo inelastic interaction in the ECAL. In order to have better accuracy, hadrons that have had high interaction with ECAL are not considered. This is ensured by requiring that the energy measured in ECAL in a cone of radius 14 cm around the impact point of the track to be less than 1 GeV.

In [ ]:
%%bash -s "$filepath"
export inputfolder=$1/CMS_POS/IsoTrack2019/data

root -l
.L testOfIsotrackPUcorrection.C 
calculateResponse("$inputfolder","dataJHT2018g",1,8,23,0,1.5,10,"test")
calculateResponse("$inputfolder","dataJHT2018g",1,8,23,1,1.5,10,"test")
.q

The above command analyzes the isolated tracks and calculates the responses for them. The results are stored in output roof file that can be accessed like below:

In [ ]:
file = ROOT.TFile("test_noCor_dataJHT2018g_const10_p40-60_pt07_eh10_ee1_rings1_1.5rms.root","read")
#file.ls()
keys = file.GetListOfKeys()

canv = ROOT.TCanvas("plots with PU", "plots with PU", 1200, 4800)
canv.Divide(4,16)

# Loop over the keys and print their names
for i, key in enumerate(keys, start=1):
    key_name = key.GetName()
    key_type = key.GetClassName()
    #print(f"Key name: {key_name}, key type: {key_type}")
    canv.cd(i)
    hist = file.Get(key.GetName())
    hist.Draw()
    
canv.Update()
canv.Draw()

In [ ]:
file = ROOT.TFile("test_cor03_dataJHT2018g_const10_p40-60_pt07_eh10_ee1_rings1_1.5rms.root","read")
#file.ls()
keys = file.GetListOfKeys()

canv = ROOT.TCanvas("plots with PU", "plots with PU", 1200, 4800)
canv.Divide(4,16)

# Loop over the keys and print their names
for i, key in enumerate(keys, start=1):
    key_name = key.GetName()
    key_type = key.GetClassName()
    #print(f"Key name: {key_name}, key type: {key_type}")
    canv.cd(i)
    hist = file.Get(key.GetName())
    hist.Draw()
    
canv.Update()
canv.Draw()


Let’s look at plots "e2pHB", "e2pHE", "e2pTR" and "e2pALL". These plots show response corrections in HB and HE.

<span style="color:black; background-color:red; font-weight:bold;">Question: Why don’t we calibrate HF in isotrack calibration method?</span>

<span style="color:black; background-color:red; font-weight:bold;">Question: What does transition region mean in the title of plot “e2pTR”?</span>


The plots show the distribution of $$\frac{E_{HCAL}}{p_{track}-E_{ECAL}}$$, which is a Gaussian. The red smooth curve is a Gaussian fit to the plot in the range -1.5 to 1.5 standard deviations. Look at the tails in the plots. Although the central part of the plot is Gaussian, the long tails on either side don’t look Gaussian. The tail on the lower side may be due to non-containment of the hadron shower in the HCAL and the tail on the higher side is most likely due to overlap of showers from some neighboring particles in the jet (most likely neutral ones).

Now look at ieta plots (from ieta -24 to ieta 24). We want to find the correction factor for each ieta separately.

<span style="color:black; background-color:red; font-weight:bold;">Question: Why do we need to find a correction factor separately for each ieta and not each iphi?</span>

The mean value of $$\frac{E_{HCAL}}{p_{track}-E_{ECAL}}$$ is our correction factor, i.e this is the factor that compensates for radiation damage. But it would be nice if we could remove long tails before calculating the mean. In order to do so, we find the center of the fitted curve instead of the mean of the whole distribution. Look at plots “respMean” and “respFit”. Can you explain what the difference is between these two plots? Look at all the plots again. Do you understand all of them? Guess what other plots are from their title. The “chi2/ndf” plot is chi-square divided by NDF (Number of Degrees of Freedom). Chi-square is a test for the goodness of a fit. What does “chi2/ndf” plot tell you?

## Correction for pile-up
### [Correcion method](https://indico.cern.ch/event/381087/contributions/905347/attachments/759198/1041419/Markin_PU_4.pdf)
When there is much energy near the cone from other tracks, there will also be contribution inside the cone, so the true energy is probably lower than that reconstructed in the cone. And if the energy deposition outside the cone is caused by the particle itself, the true energy will be higher than that reconstructed in the cone. We use a 70-cm wide (Rcone = 35 cm) cone when measuring the energy of particles. If there is any energy deposition from a nearby track in our 70-cm wide cone, the energy beyond the cone (radius > Rcone) should be higher than normal.

It is found that looking at the energy deposition between Rcone + 10 cm and Rcone + 30 cm has better results because we still have some energy from the selected isolated track between Rcone and Rcone + 10 cm. We show the energy between Rcone + 10 cm and Rcone + 30 cm by ∆.

<img src="./notebookplots/delta4PUcorrection.png " alt="Alt Text" width="80%" height="80%">

### Compare MC samples with and without pile-up

In [ ]:
Chain_noPU = ROOT.TChain("CalibTree")
Chain_noPU.Add(filepath+"CMS_POS/IsoTrack2019/mc/mc2018noPUnew.root")
for branch in Chain_noPU.GetListOfBranches(): print(branch.GetName())

Hist_noPU = ROOT.TProfile("no_PU", ";ieta;#Delta (GeV)", 60, -30, 30)

for i, event in tqdm(enumerate(Chain_noPU)): 
    if i>10000: break
    if abs(event.t_ieta) > 25: continue
    if event.t_selectTk and event.t_qltyMissFlag and event.t_qltyPVFlag:
        Hist_noPU.Fill(event.t_ieta, event.t_eHcal30 - event.t_eHcal10)


        
Chain_PU = ROOT.TChain("CalibTree")
Chain_PU.Add(filepath+"CMS_POS/IsoTrack2019/mc/mc2018PUnew.root")
#for branch in Chain_noPU.GetListOfBranches(): print(branch.GetName())

Hist_PU = ROOT.TProfile("PU", ";ieta;#Delta (GeV)", 60, -30, 30)

for i, event in tqdm(enumerate(Chain_PU)): 
    if i>10000: break
    if abs(event.t_ieta) > 25: continue
    if event.t_selectTk and event.t_qltyMissFlag and event.t_qltyPVFlag:
        Hist_PU.Fill(event.t_ieta, event.t_eHcal30 - event.t_eHcal10)


c1 = ROOT.TCanvas("MC_Data","",900,600)

leg = ROOT.TLegend(0.8,0.85,0.95,0.95)
leg.AddEntry(Hist_noPU, "no PU")
leg.AddEntry(Hist_PU, "with PU")

Hist_noPU.SetMaximum(10)
Hist_noPU.SetMinimum(0)
Hist_noPU.SetLineColor(ROOT.kBlue)
Hist_noPU.SetStats(0)
Hist_noPU.Draw()

Hist_PU.SetLineColor(ROOT.kRed)
Hist_PU.Draw("same")

leg.Draw()
c1.Update()
c1.Draw()


### Correction Formula
In order to make up for PU, we correct the energy measured by HCAL using this formula:

<img src="notebookplots/parametrisation2019.png " alt="Alt Text" width="60%" height="60%">  
<img src="./notebookplots/parameters2018.png " alt="Alt Text" width="60%" height="60%">

Where E is the energy in the signal cone Rcone = 35 cm around the impact point of the selected isolated track, p is the track momentum and ∆ is the energy deposition in the annulus around the main cone as explained above. The coefficients are determined by asking the energy measured by HCAL be equal to the track momentum minus ECAL energy (in MC)

The quadratic term (third term) can be explained by the fact that there is also some fraction of pion energy outside the cone, in particular for low-energy showers that tend to be wider. Accounting for them works in opposite direction and thus lessens the correlation between the reconstructed pion energy and pileup energy deposited near the cone. Therefore, the correction for low-energy showers should be suppressed with respect to high-energy ones.